In [79]:
import pandas as pd

In [80]:
import numpy as np

## 1. Data Loadind

In [81]:
jogos = pd.read_csv("data/match-results.7.csv")

In [82]:
del jogos["Unnamed: 0"]

In [83]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado
0,brasileiro-a,2018,8,31-05,Atlético-PR,Santos,2,0,MANDANTE
1,brasileiro-a,2018,8,31-05,Corinthians,América-MG,1,0,MANDANTE
2,brasileiro-a,2018,8,31-05,Flamengo,Bahia,2,0,MANDANTE
3,brasileiro-a,2018,8,30-05,Cruzeiro,Palmeiras,1,0,MANDANTE
4,brasileiro-a,2018,8,30-05,Grêmio,Fluminense,0,0,EMPATE


In [84]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6204 entries, 0 to 6203
Data columns (total 9 columns):
campeonato        6204 non-null object
ano               6204 non-null int64
rodada            6204 non-null int64
data              6204 non-null object
mandante          6204 non-null object
visitante         6204 non-null object
gols_mandante     6204 non-null int64
gols_visitante    6204 non-null int64
resultado         6204 non-null object
dtypes: int64(4), object(5)
memory usage: 436.3+ KB


In [85]:
times = list(jogos["mandante"].unique())

## 2. Checking results 

### 2.1 Results grouped

In [86]:
totais_resultados = jogos.groupby("resultado").size()

### 2.2 Total  of matches

In [87]:
total_jogos = totais_resultados.sum()

### 2.3 Create a DataFrame with grouped results

In [88]:
resultados_DF = pd.DataFrame({'resultados': totais_resultados.keys(), 'totais': totais_resultados.values})

In [89]:
resultados_DF

,resultados,totais
0,EMPATE,1609
1,MANDANTE,3144
2,VISITANTE,1451


In [90]:
def percentual(valor, total):
    return round(valor/total, 4)

In [91]:
resultados_DF['%'] = resultados_DF.apply (lambda row: percentual(row["totais"], total_jogos),axis=1)

In [92]:
resultados_DF

,resultados,totais,%
0,EMPATE,1609,0.2593
1,MANDANTE,3144,0.5068
2,VISITANTE,1451,0.2339


### Calcular resultados de um time

In [93]:
def resultadosTime(time, casa, resultado):
    return len(jogos[(jogos[casa] == time) & (jogos["resultado"] == resultado)]) 

In [94]:
resultados_time_DF = pd.DataFrame({'time': times })

In [95]:
resultados_time_DF["vitorias_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "MANDANTE"),axis=1)
resultados_time_DF["empates_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "VISITANTE"),axis=1)
resultados_time_DF["vitorias_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "VISITANTE"),axis=1)
resultados_time_DF["empates_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "MANDANTE"),axis=1)

In [96]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora
0,Atlético-PR,159,73,48,60,60,160
1,Corinthians,159,69,52,88,88,104
2,Flamengo,151,89,59,80,86,133


In [97]:
resultados_time_DF.eval("jogos_casa = vitorias_casa + empates_casa + derrotas_casa", inplace=True)
resultados_time_DF.eval("jogos_fora = vitorias_fora + empates_fora + derrotas_fora", inplace=True)
resultados_time_DF.eval("jogos_total = jogos_fora + jogos_fora", inplace=True)

In [98]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total
0,Atlético-PR,159,73,48,60,60,160,280,280,560
1,Corinthians,159,69,52,88,88,104,280,280,560
2,Flamengo,151,89,59,80,86,133,299,299,598


In [99]:
def aproveitamentoMandante(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    return (3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante)

In [100]:
resultados_time_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: aproveitamentoMandante(row),axis=1)
resultados_time_DF.head(3)


,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa
0,Atlético-PR,159,73,48,60,60,160,280,280,560,0.654762
1,Corinthians,159,69,52,88,88,104,280,280,560,0.650000
2,Flamengo,151,89,59,80,86,133,299,299,598,0.604236


In [101]:
def aproveitamentoVisitante(row):
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"]) / (3*total_jogos_visitante)

In [102]:
resultados_time_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: aproveitamentoVisitante(row),axis=1)
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora
0,Atlético-PR,159,73,48,60,60,160,280,280,560,0.654762,0.285714
1,Corinthians,159,69,52,88,88,104,280,280,560,0.650000,0.419048
2,Flamengo,151,89,59,80,86,133,299,299,598,0.604236,0.363434


In [103]:
def aproveitamentoTotal(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"] + 3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante + 3*total_jogos_visitante)

In [104]:
resultados_time_DF["aprov_total"] = resultados_time_DF.apply (lambda row: aproveitamentoTotal(row),axis=1)
resultados_time_DF.sort_values(["aprov_total"], ascending=[0]).head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total
6,São Paulo,179,73,47,101,80,118,299,299,598,0.680045,0.426979,0.553512
1,Corinthians,159,69,52,88,88,104,280,280,560,0.650000,0.419048,0.534524
3,Cruzeiro,171,67,61,105,59,135,299,299,598,0.646600,0.416945,0.531773


In [105]:
resultados_time_DF = resultados_time_DF[["time", "jogos_casa", "jogos_fora", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total"]].sort_values(["aprov_total"], ascending=[0])

In [106]:
def golsMandante(time):
    return jogos[jogos["mandante"] == time][["gols_mandante"]].sum()

In [107]:
def golsVisitante(time):
    return jogos[jogos["visitante"] == time]["gols_visitante"].sum()

In [108]:
resultados_time_DF["gols_mandante"] = resultados_time_DF.apply (lambda row: golsMandante(row["time"]),axis=1)
resultados_time_DF["gols_visitante"] = resultados_time_DF.apply (lambda row: golsVisitante(row["time"]),axis=1)

resultados_time_DF.eval("total_gols = gols_mandante + gols_visitante", inplace=True)
resultados_time_DF.eval("media_gols_casa = gols_mandante/jogos_casa", inplace=True)
resultados_time_DF.eval("media_gols_fora = gols_visitante/jogos_fora", inplace=True)
resultados_time_DF.eval("media_gols = (gols_mandante + gols_visitante)/jogos_total", inplace=True)

In [109]:
resultados_time_DF.head()


,time,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total,gols_mandante,gols_visitante,total_gols,media_gols_casa,media_gols_fora,media_gols
6,São Paulo,299,299,598,0.680045,0.426979,0.553512,536,370,906,1.792642,1.237458,1.515050
1,Corinthians,280,280,560,0.650000,0.419048,0.534524,442,316,758,1.578571,1.128571,1.353571
3,Cruzeiro,299,299,598,0.646600,0.416945,0.531773,556,366,922,1.859532,1.224080,1.541806
17,Santos,298,299,598,0.673378,0.360089,0.516471,551,357,908,1.848993,1.193980,1.518395
16,Internacional,279,280,560,0.663082,0.360714,0.511628,446,299,745,1.598566,1.067857,1.330357


In [110]:
resultados_time_DF = resultados_time_DF[["time", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total", "media_gols_casa", "media_gols_fora", "media_gols"]]

In [111]:
resultados_time_DF.set_index("time", inplace=True)

In [112]:
resultados_time_DF.sort_values(["aprov_total"], ascending = [0]).head(3)

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
time,,,,,,,
São Paulo,598,0.680045,0.426979,0.553512,1.792642,1.237458,1.515050
Corinthians,560,0.650000,0.419048,0.534524,1.578571,1.128571,1.353571
Cruzeiro,598,0.646600,0.416945,0.531773,1.859532,1.224080,1.541806


In [113]:
resultados_time_DF_DESC = resultados_time_DF.describe(percentiles=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

In [114]:
resultados_time_DF_DESC

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,302.634146,0.547644,0.269519,0.408592,1.501933,0.961233,1.231715
std,209.521092,0.095963,0.089347,0.083671,0.244847,0.187919,0.192109
min,38.000000,0.157895,0.087719,0.149123,0.631579,0.368421,0.631579
0%,38.000000,0.157895,0.087719,0.149123,0.631579,0.368421,0.631579
10%,42.000000,0.438596,0.140351,0.307018,1.261905,0.738095,1.023810
20%,84.000000,0.500000,0.198413,0.350877,1.309524,0.872928,1.131579
30%,134.000000,0.508772,0.210526,0.375661,1.408840,0.894737,1.166667
40%,172.000000,0.538813,0.265193,0.383085,1.466387,0.947368,1.210046
50%,218.000000,0.555556,0.270308,0.417460,1.553719,1.000000,1.230159


In [115]:
percentis = resultados_time_DF_DESC.loc[["10%","20%","30%","40%","50%","60%","70%","80%","90%"]]

In [116]:
percentis

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
10%,42.0,0.438596,0.140351,0.307018,1.261905,0.738095,1.023810
20%,84.0,0.500000,0.198413,0.350877,1.309524,0.872928,1.131579
30%,134.0,0.508772,0.210526,0.375661,1.408840,0.894737,1.166667
40%,172.0,0.538813,0.265193,0.383085,1.466387,0.947368,1.210046
50%,218.0,0.555556,0.270308,0.417460,1.553719,1.000000,1.230159
60%,366.0,0.580825,0.287462,0.431866,1.578947,1.010791,1.289474
70%,482.0,0.595129,0.330739,0.455253,1.598566,1.065789,1.332685
80%,560.0,0.608333,0.357143,0.482738,1.703571,1.099585,1.362385
90%,598.0,0.654762,0.373541,0.511628,1.776256,1.164286,1.440299


In [117]:
def get_stars(array,value):
    print ()
    idx = (np.abs(array-value)).argmin()
    near = array[idx][0]
    index = np.where(array==near)[0][0]
    
    if (value > near):
        index = index + 1  
       
    return index + 1

In [118]:
stars_DF = resultados_time_DF.copy()

In [119]:
stars_DF["aprov_total"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_total"]].values, row["aprov_total"]),axis=1)

In [120]:
stars_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_casa"]].values, row["aprov_casa"]),axis=1)

In [121]:
stars_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_fora"]].values, row["aprov_fora"]),axis=1)

In [122]:
stars_DF["media_gols_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_casa"]].values, row["media_gols_casa"]),axis=1)

In [123]:
stars_DF["media_gols_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_fora"]].values, row["media_gols_fora"]),axis=1)

In [124]:
stars_DF["media_gols"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols"]].values, row["media_gols"]),axis=1)

In [130]:
stars_DF.eval("forca_casa = (3*aprov_casa + media_gols_casa)/4", inplace=True)

In [131]:
stars_DF.eval("forca_fora = (3*aprov_fora + media_gols_fora)/4", inplace=True)

In [132]:
stars_DF.eval("forca = (3*aprov_total + 1*media_gols)/4", inplace=True)

In [133]:
stars_DF.sort_values(["forca"], ascending=[0])

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols,forca_casa,forca_fora,forca
time,,,,,,,,,,
São Paulo,598,10,10,10,10,10,10,10.00,10.00,10.00
Santos,598,10,9,10,10,10,10,10.00,9.25,10.00
Cruzeiro,598,9,10,10,10,10,10,9.25,10.00,10.00
Corinthians,560,9,10,10,6,9,8,8.25,9.75,9.50
Palmeiras,514,9,9,9,8,9,9,8.75,9.00,9.00
Grêmio,556,10,8,9,9,6,8,9.75,7.50,8.75
Internacional,560,10,9,9,7,8,7,9.25,8.75,8.50
Flamengo,598,8,9,9,6,7,7,7.50,8.50,8.50
Atlético-MG,560,8,8,8,9,9,10,8.25,8.25,8.50


In [134]:
stars_DF.to_csv(path_or_buf="data/team_stars.8.csv",sep=";")